In [252]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import ensemble, tree, linear_model
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.utils import shuffle
from pandas import read_csv
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
import matplotlib
from matplotlib import pyplot
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [253]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train_ids = train['ID']
train = train.drop('ID', axis=1)
test_ids = test['ID']
test = test.drop('ID', axis=1)

targets = train['y']
train = train.drop('y', axis=1)
print("Train data shape: ", train.shape)
print("Test data shape: ", test.shape)
print("Targets data shape: ", targets.shape)

Train data shape:  (4209, 376)
Test data shape:  (4209, 376)
Targets data shape:  (4209,)


In [254]:
df = pd.concat([train,test], axis=0)
df.index = range(0,8418)
df.shape

(8418, 376)

In [255]:
print(df.head())

   X0 X1  X2 X3 X4 X5 X6 X8  X10  X11  ...   X375  X376  X377  X378  X379  \
0   k  v  at  a  d  u  j  o    0    0  ...      0     0     1     0     0   
1   k  t  av  e  d  y  l  o    0    0  ...      1     0     0     0     0   
2  az  w   n  c  d  x  j  x    0    0  ...      0     0     0     0     0   
3  az  t   n  f  d  x  l  e    0    0  ...      0     0     0     0     0   
4  az  v   n  f  d  h  d  n    0    0  ...      0     0     0     0     0   

   X380  X382  X383  X384  X385  
0     0     0     0     0     0  
1     0     0     0     0     0  
2     0     1     0     0     0  
3     0     0     0     0     0  
4     0     0     0     0     0  

[5 rows x 376 columns]


In [256]:
le = LabelEncoder()

X0 = df['X0'].T
X1 = df['X1'].T
X2 = df['X2'].T
X3 = df['X3'].T
X4 = df['X4'].T
X5 = df['X5'].T
X6 = df['X6'].T
X8 = df['X8'].T

len(list(X0.unique()))

53

In [257]:
le.fit(X0)
X0 = le.transform(X0)
X0 = pd.get_dummies(X0)

le.fit(X1)
X1 = le.transform(X1)
X1 = pd.get_dummies(X1)

le.fit(X2)
X2 = le.transform(X2)
X2 = pd.get_dummies(X2)

le.fit(X3)
X3 = le.transform(X3)
X3 = pd.get_dummies(X3)

le.fit(X4)
X4 = le.transform(X4)
X4 = pd.get_dummies(X4)

le.fit(X5)
X5 = le.transform(X5)
X5 = pd.get_dummies(X5)

le.fit(X6)
X6 = le.transform(X6)
X6 = pd.get_dummies(X6)

le.fit(X8)
X8 = le.transform(X8)
X8 = pd.get_dummies(X8)

In [258]:
categoricals = pd.concat([X0, X1, X2, X3, X4, X5, X6, X8], axis=1)
print(categoricals.shape)

(8418, 211)


In [259]:
df = df.drop(['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8'], axis=1)

In [260]:
df_ = pd.get_dummies(df)

In [261]:
print(df_.shape)
print(categoricals.shape)
print(targets.shape)

(8418, 368)
(8418, 211)
(4209,)


In [262]:
df = pd.concat([categoricals, df_], axis=1)
print("Total dataset shape: ", df.shape)

Total dataset shape:  (8418, 579)


In [263]:
train = df[0:4209]
test = df[4209:]
print(train.shape)
print(test.shape)

(4209, 579)
(4209, 579)


In [264]:
x_train, x_test, y_train, y_test = train_test_split(train, targets, random_state=42)
x_train, x_test, y_train, y_test = x_train.values, x_test.values, y_train.values, y_test.values

In [265]:
eval_set = [(x_test, y_test)]
model = XGBRegressor(n_estimators=1000, max_depth=2)
model.fit(x_train, y_train, eval_metric="rmse", eval_set=eval_set, early_stopping_rounds=10, verbose=False)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=2,
       min_child_weight=1, missing=None, n_estimators=1000, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [273]:
from sklearn.metrics import mean_squared_error, r2_score
y_pred = model.predict(x_test)
score = mean_squared_error(y_test, y_pred)
print("RMSE on x_test: ", np.sqrt(score))
print("R2 on x_test: ", r2_score(y_test, y_pred))

RMSE on x_test:  8.18402640469
R2 on x_test:  0.578581860023


In [271]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
kfold = KFold(n_splits=10, random_state=7)
results = cross_val_score(model, x_train, y_train, cv=kfold)
print("Train cross val R2: ", results.mean())

Train cross val R2:  0.540503338401


In [272]:
kfold = KFold(n_splits=10, random_state=7)
results = cross_val_score(model, x_test, y_test, cv=kfold)
print("Test cross val R2: ", results.mean())

Test cross val R2:  0.5222533399


In [274]:
sub = model.predict(test.values)
sub = pd.DataFrame(sub)
sub.columns = ['y']
ID = pd.DataFrame(ids[4209:].values)
ID.columns = ['ID']
sub = pd.concat([ID, sub], axis=1)
sub.head(5)

,ID,y
0,1,78.703758
1,2,94.309814
2,3,78.703758
3,4,78.703758
4,5,111.763451


In [270]:
sub.to_csv('./final_xgsubmission.csv', index=False)